# COPY INTO and MERGE Commands

COPY INTO Command
* Incrementally load data into Delta Lake tables from cloud storage
* Support schema evolution
* Support wide range of file formats (CSV, JSON, Parquet, Delta)
* Alternative to Auto Loader for batch ingestion 

Documentation - [COPY INTO](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/delta-copy-into)

Create the table to copy data into

In [0]:
CREATE TABLE IF NOT EXISTS delta_lake.raw_stock_prices;

In [0]:
%python
SAS_TOKEN = "<sas_token>"
STORAGE_ACCOUNT_NAME = 'udemyttstorage'

spark.conf.set(f"fs.azure.account.auth.type.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", SAS_TOKEN)

In [0]:
TRUNCATE TABLE delta_lake.raw_stock_prices;

COPY INTO delta_lake.raw_stock_prices
FROM 'abfss://demo@udemyttstorage.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

Incrementally load new files into the table

In [0]:
SELECT * FROM delta_lake.raw_stock_prices;

MERGE INTO Statement
* Used for upserts (Insert/Update/Delete operations in a single statement)
* Allows merging new dta into target table based on matching conditions

Documentation - [MERGE INTO](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/delta-merge-into)

Create the table to merge data into

In [0]:
CREATE OR REPLACE TABLE delta_lake.stock_prices(
  stock_id STRING,
  price DOUBLE,
  trading_date DATE
);

Merge the source data into the target table
1. Insert new stocks received
2. Update price and trading_date if updates received
3. Delete stocks which are de-listed from the exchange (status = 'DELISTED')

In [0]:
MERGE INTO delta_lake.stock_prices as t
USING delta_lake.raw_stock_prices as s
ON t.stock_id = s.stock_id
WHEN MATCHED AND s.status = 'ACTIVE' THEN
  UPDATE SET
    t.price = s.price,
    t.trading_date = s.trading_date
WHEN MATCHED AND s.status = 'DELISTED' THEN
  DELETE
WHEN NOT MATCHED AND s.status = 'ACTIVE' THEN
  INSERT (stock_id, price, trading_date)
  VALUES (s.stock_id, s.price, s.trading_date);

In [0]:
SELECT * FROM delta_lake.stock_prices;